In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

my_data=pd.read_csv('random_data.csv')
my_data_refined=my_data.drop(['Id'], axis=1)
print(my_data_refined.head())

list_=np.zeros((my_data_refined.shape[0],my_data_refined.shape[1]-1)) #Shape[0] gives number of rows
list_[:, 0]=my_data_refined["x1"]
list_[:, 1]=my_data_refined["x2"]
list_[:, 2]=my_data_refined["x3"]
list_[:, 3]=my_data_refined["x4"]
target=np.zeros((my_data_refined.shape[0],1))

#print(len(list_))
target[:,0]=my_data_refined["y"]
print(len(target))
def Data_Normalization(X):
  mean=np.mean(X, axis=0)
  sd= np.std(X, axis=0)
  
  N=X.shape[0] #Number of Examples

  # insert extra singleton dimension, to obtain 1xD shape i.e to bypass the shape (4,): we want (1,4)

  mean_vector = np.expand_dims(mean, axis=0)
  std_vector = np.expand_dims(sd, axis=0)
  
  #Repeat N times to get NxD shape
  Ndim_mean = np.repeat(mean_vector, N, axis=0)
  Ndim_std = np.repeat(std_vector, N, axis=0)

  if(np.any(Ndim_std)==0):
    Ndim_std+=.000000001

  X_Normalized=(X-Ndim_mean)/Ndim_std

  return(X_Normalized, mean_vector, std_vector)





In [ ]:
import random
data, mu, sigma = Data_Normalization(list_) #Getting the normalized data
bias_column=np.ones((data.shape[0],1))      #creating the bias term
data=np.append(bias_column,data, axis=1)    #adding the bias term
theta=np.zeros(data.shape[1])               #Creating a rank 1 array


In [ ]:
def calculate_hypothesis(data,theta,i):
  h_theta=np.dot(theta,data[i])
  return(h_theta)


In [ ]:
def calculate_theta(data,y,learning_rate,theta,max_iteration):

  cost_vector=np.array([], dtype=np.float32)
  no_of_examples=data.shape[0]                      #Number of Examples extracted
  for i in range (max_iteration):                   #Number of Epoch

    theta_temp=theta.copy()
    sum_over_del_j_theta_d_theta_k=np.zeros(no_of_examples)
    for j in range (no_of_examples):                #For all examples

      hypothesis=calculate_hypothesis(data,theta,j) #for a particular example calculating prediction

      for k in range (len(theta)):                  #For all theta and all examples we are calculating the sum_over_del_j_theta_d_theta_k
        sum_over_del_j_theta_d_theta_k[k] +=(hypothesis-y[j])*data[j,k]    #jth row of kth example :data[j,k] 

    for k in range (len(theta)):                    #Calculating Theta : to Update Simultaneously
      theta_temp[k]= theta_temp[k] - ((learning_rate)/no_of_examples)*sum_over_del_j_theta_d_theta_k[k]
    
    theta=theta_temp.copy()                         #Update Simultaneously

    cost_after_this_iteration = calculate_cost(data,y,theta) #Calculating cost for visual input
    print("After iteration {} cost is {:f}".format(i,float(cost_after_this_iteration)))
    cost_vector = np.append(cost_vector, cost_after_this_iteration)

    if (cost_after_this_iteration<.0001):
      break
  plot_cost(cost_vector)
  return (theta)



In [ ]:
def calculate_cost(data,y,theta):
  no_of_training_examples=y.shape[0]
  avg_cost=0
  sum=0
  for i in range (no_of_training_examples):
    prediction=calculate_hypothesis(data,theta,i)
    
    target=y[i]
    j_theta= (target-prediction)**2
    
    sum+=j_theta
    #print(i)
  avg_cost=sum/(2*no_of_training_examples)
  
  return avg_cost
  


In [ ]:
def plot_cost(cost):
  figure,axis=plt.subplots()
  axis.set_xlabel('Itearion')
  axis.set_ylabel('Cost')
  plt.plot(cost)
  plt.show()

In [ ]:
def exact_solution(data,target):
  theta=np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(data),data)),np.transpose(data)),target)
  Theta_=theta.ravel()  #ND array to 1D list
  return Theta_

In [ ]:
Theta=calculate_theta(data,target,.1,theta,1000)
Theta_=exact_solution(data,target)
print(Theta)
print(Theta_)

In [ ]:
data_=np.array([[85, 2, 15, -0.170014769]])
data_=(data_-mu)/sigma
column_of_ones = np.ones((data_.shape[0], 1))
data_with_bias = np.append(column_of_ones, data_, axis=1)
print("Prediction using Gradient Descent is ",calculate_hypothesis(data_with_bias,Theta,0))
print("Prediction using linear Algebra is ",calculate_hypothesis(data_with_bias,Theta_,0))

Prediction using Gradient Descent is  296.35714605450977
Prediction using linear Algebra is  296.3680298844417
